In [1]:
import cv2
import numpy as np
from Cryptodome.Cipher import AES
import hashlib, binascii

def embed(file_path, video_path, output_path, password, channels=3):
    with open(file_path, "rb") as f:
        data = f.read()

    if not data:
        raise ValueError("File is empty.")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Couldn't open video.")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30  # fallback if undefined

    # If metadata isn't available, read manually
    if frame_count <= 0 or width <= 0 or height <= 0:
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        frame_count = len(frames)
        if frame_count == 0:
            raise ValueError("Video has no frames.")
        height, width = frames[0].shape[:2]
        cap.release()
        cap = cv2.VideoCapture(video_path)

    capacity = frame_count * width * height * channels
    data_len = len(data)

    # Hash password → AES key
    aes_key = hashlib.sha256(password.encode()).digest()[:16]
    cipher = AES.new(aes_key, AES.MODE_ECB)

    # Metadata: file size, CRC, and a "magic" marker
    metadata = (
        data_len.to_bytes(8, 'little') +
        binascii.crc32(data).to_bytes(4, 'little', signed=False) +
        b'STEG'
    )

    if len(metadata) != 16:
        raise ValueError("Bad metadata length")

    encrypted_meta = cipher.encrypt(metadata)

    # Prepare bits to hide
    meta_bits = np.unpackbits(np.frombuffer(encrypted_meta, dtype=np.uint8))
    data_bits = np.unpackbits(np.frombuffer(data, dtype=np.uint8))

    # Optional marker at the end
    marker_bits = np.unpackbits(np.frombuffer(b'\x00\xFF\x00\xFF', dtype=np.uint8))
    bits_to_hide = np.concatenate([meta_bits, data_bits, marker_bits])

    if bits_to_hide.size > capacity:
        raise ValueError("Not enough room in video to embed data.")

    # Pseudo-random bit positions from AES-PRNG
    total_bits = bits_to_hide.size
    used = set()
    indices = []
    counter = 0
    prng = AES.new(aes_key, AES.MODE_ECB)

    while len(indices) < total_bits:
        block = prng.encrypt(counter.to_bytes(16, 'little'))
        counter += 1
        for i in range(0, 16, 4):
            if len(indices) >= total_bits:
                break
            pos = int.from_bytes(block[i:i+4], 'little') % capacity
            if pos not in used:
                used.add(pos)
                indices.append(pos)

    indices = np.array(indices, dtype=np.uint64)
    frame_size = width * height * channels
    frame_bits = indices // frame_size
    pixel_bits = indices % frame_size

    mapping = {}
    for i, frame_idx in enumerate(frame_bits):
        mapping.setdefault(int(frame_idx), []).append(i)

    # Use FFV1 for lossless output
    out_ext = output_path.split('.')[-1].lower()
    if out_ext not in ('mkv', 'avi'):
        output_path = (output_path.rsplit('.', 1)[0] if '.' in output_path else output_path) + '.mkv'

    writer = cv2.VideoWriter(
        output_path,
        cv2.VideoWriter_fourcc(*'FFV1'),
        fps,
        (width, height)
    )

    if not writer.isOpened():
        cap.release()
        raise ValueError("Couldn't create output video.")

    for frame_idx in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx in mapping:
            bit_idxs = mapping[frame_idx]
            pixels = frame.reshape(-1)
            pixel_pos = pixel_bits[bit_idxs]
            bit_vals = bits_to_hide[bit_idxs].astype(np.uint8)
            pixels[pixel_pos] = (pixels[pixel_pos] & ~1) | bit_vals
            frame = pixels.reshape(frame.shape)

        writer.write(frame)

    cap.release()
    writer.release()
    return output_path


In [ ]:
import cv2
import numpy as np
from Cryptodome.Cipher import AES
import hashlib, binascii

def extract(video_path, output_path, password, channels=3):
    # Try to open the video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Couldn't open the video.")

    # Get video properties
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Fallback if video properties are unavailable
    if frame_count <= 0 or width <= 0 or height <= 0:
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        frame_count = len(frames)
        if frame_count == 0:
            raise ValueError("Video seems to be empty.")
        height, width = frames[0].shape[:2]
        cap.release()
        cap = cv2.VideoCapture(video_path)

    frame_size = width * height * channels
    total_capacity = frame_count * frame_size

    # Derive AES key from password
    aes_key = hashlib.sha256(password.encode()).digest()[:16]
    cipher = AES.new(aes_key, AES.MODE_ECB)

    # Extract the first 128 bits for metadata
    metadata_bits = 128
    used = set()
    meta_positions = []
    counter = 0
    prng = AES.new(aes_key, AES.MODE_ECB)

    while len(meta_positions) < metadata_bits:
        block = prng.encrypt(counter.to_bytes(16, 'little'))
        counter += 1
        for i in range(0, 16, 4):
            if len(meta_positions) >= metadata_bits:
                break
            idx = int.from_bytes(block[i:i+4], 'little') % total_capacity
            if idx not in used:
                used.add(idx)
                meta_positions.append(idx)

    meta_positions = np.array(meta_positions, dtype=np.uint64)
    meta_frames = meta_positions // frame_size
    meta_pixel_indices = meta_positions % frame_size

    meta_bits = np.zeros(metadata_bits, dtype=np.uint8)
    max_meta_frame = int(meta_frames.max()) if meta_frames.size else -1

    for f_idx in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        mask = (meta_frames == f_idx)
        if np.any(mask):
            flat = frame.reshape(-1)
            bits = flat[meta_pixel_indices[mask]] & 1
            meta_bits[np.where(mask)[0]] = bits
        if f_idx >= max_meta_frame:
            break

    # Decrypt and parse metadata
    decrypted = cipher.decrypt(np.packbits(meta_bits).tobytes())
    file_size = int.from_bytes(decrypted[:8], 'little')
    expected_crc = int.from_bytes(decrypted[8:12], 'little')
    magic = decrypted[12:16]

    if magic != b'STEG':
        raise ValueError("Bad password or corrupted data.")

    data_bits_count = file_size * 8
    marker = np.unpackbits(np.frombuffer(b'\x00\xFF\x00\xFF', dtype=np.uint8))
    use_marker = True
    marker_bits_count = marker.size if use_marker else 0
    total_bits_needed = 128 + data_bits_count + marker_bits_count

    if total_bits_needed > total_capacity:
        raise ValueError("Video doesn't have enough hidden data.")

    # Generate all bit positions
    used_all = set()
    all_positions = []
    counter = 0

    while len(all_positions) < total_bits_needed:
        block = prng.encrypt(counter.to_bytes(16, 'little'))
        counter += 1
        for i in range(0, 16, 4):
            if len(all_positions) >= total_bits_needed:
                break
            idx = int.from_bytes(block[i:i+4], 'little') % total_capacity
            if idx not in used_all:
                used_all.add(idx)
                all_positions.append(idx)

    all_positions = np.array(all_positions, dtype=np.uint64)
    all_frames = all_positions // frame_size
    all_pixel_indices = all_positions % frame_size

    # Rewind video and extract all bits
    cap.release()
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Couldn’t reopen video for extraction.")

    hidden_bits = np.zeros(total_bits_needed, dtype=np.uint8)
    max_data_frame = int(all_frames.max())

    for f_idx in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        mask = (all_frames == f_idx)
        if np.any(mask):
            flat = frame.reshape(-1)
            bits = flat[all_pixel_indices[mask]] & 1
            hidden_bits[np.where(mask)[0]] = bits
        if f_idx >= max_data_frame:
            break
    cap.release()

    payload_bits = hidden_bits[128:]  # skip metadata bits

    # Check for marker and isolate real data bits
    if use_marker:
        if payload_bits.size < marker_bits_count:
            raise ValueError("No marker found.")
        extracted_data_bits = payload_bits[:data_bits_count]
        extracted_marker = payload_bits[data_bits_count:data_bits_count + marker_bits_count]
        if not np.array_equal(extracted_marker, marker):
            pass  # could warn or raise, but we'll proceed anyway
    else:
        extracted_data_bits = payload_bits[:data_bits_count]

    extracted_data = np.packbits(extracted_data_bits).tobytes()

    # Validate CRC
    if binascii.crc32(extracted_data) & 0xFFFFFFFF != expected_crc:
        raise ValueError("CRC mismatch — data may be corrupted.")

    with open(output_path, "wb") as out_file:
        out_file.write(extracted_data)

    return file_size


In [ ]:
file_to_embed = input("Enter the file to embed: ")
video_to_embed_in = input("Enter the video file to embed in: ")
output_video = input("Enter the output video file: ")
#password bust 50 characters long
password = input("Enter the password: ")
if len(password) > 50:
    password = password[:50]
# Embed the file
embed(file_to_embed, video_to_embed_in, output_video, password)

In [ ]:
video_to_embed_in = input("Enter the video file to extract from: ")
file_to_extracted = input("Enter the output file: ")
#password bust 50 characters long
password = input("Enter the password: ")
if len(password) > 50:
    password = password[:50]
# Extract the file
extracted_size = extract(video_to_embed_in, file_to_extracted, password)